In [ ]:
import os
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import h5py
import numpy as np
import yaml

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Arial",
    "font.size": 15,
})

script_dir = os.getcwd()
# ---------- here is the path to the results directories ----------------------
mqlib_results_dir_path = f"{script_dir}/outputs/qbp/very_large/2024-05-18/11-00-16"
annealing_results_dirs = f"{script_dir}/outputs/qbp/very_large/"
# ---------------------------------------------------------------------------
simcim_results_path = f"{mqlib_results_dir_path}/simcim_result"
mqlib_results_path = f"{mqlib_results_dir_path}/mqlib_result"
annealing_results_dirs_list = os.walk(annealing_results_dirs).__next__()[1]

total_evolution_times = []
annealing_energies = []
for annealing_results_dir in annealing_results_dirs_list:
    annealing_results_dir = f"{annealing_results_dirs}{annealing_results_dir}"
    subresults_dirs = os.walk(annealing_results_dir).__next__()[1]
    for subresults_dir in subresults_dirs:
        final_dir = f"{annealing_results_dir}/{subresults_dir}/result"
        config_path = f"{annealing_results_dir}/{subresults_dir}/.hydra/config.yaml"
        with open(config_path, "r") as f:
            raw_config = f.read()
            config = yaml.safe_load(raw_config)
            time_steps_num =int(config["quantum_annealing_schedule"]["steps_number"])
            time_step_size = float(config["quantum_annealing_schedule"]["total_time_step_size"])
            total_evolution_times.append(time_steps_num * time_step_size)
        with h5py.File(final_dir, "r") as f:
            annealing_energies.append(np.array(f["quantum_annealer_energy_wo_sampling"]))
            pairs = np.array(f["coupled_spin_pairs"])
            edges_number = pairs.shape[0]
time_indices = np.argsort(total_evolution_times)
total_evolution_times = np.array(total_evolution_times)[time_indices]
annealing_energies = np.array(annealing_energies)[time_indices]
with h5py.File(simcim_results_path, "r") as f:
    simcim_energy = np.array(f["simcim_energy"])
heuristics_energy = [simcim_energy]
heuristics_names = ["SIMCIM"]
with h5py.File(mqlib_results_path, "r") as f:
    for key in f.keys():
        if "energy" in key:
            energy = np.array(f[key])
            heuristics_energy.append(energy)
            heuristics_names.append(key.split("_")[0])
name_and_energy = list(zip(heuristics_names, heuristics_energy))
name_and_energy.sort(key=lambda x: x[1])
sorted_energies = list(map(lambda x: -x[1], name_and_energy))
fig_mqlib_hierarchy, ax = plt.subplots(1, 1)
ax.scatter(
    np.arange(len(name_and_energy)),
    sorted_energies,
    color="b",
    label=r"$E:\ {\rm heuristics}$"
)
ax.set_ylim(bottom = 200, top = 1520)
ax.set_xticks(np.arange(len(name_and_energy)), list(map(lambda x: x[0], name_and_energy)), rotation=90, fontsize=9, family="monospace")
ax.axhline(-min(annealing_energies), color='k', label=r"$E:\ {\rm quantum \ annealing \ simulation}$")
ax.text(27, -min(annealing_energies) + 22, r"GTQA ($T = 640$)", fontsize=13)
for i in np.arange(len(name_and_energy)):
    ax.plot([i, i], [0, sorted_energies[i]], color="blue", linestyle='dotted', linewidth=0.7, alpha = 0.6)
ax.set_ylabel(r"$E$")
cmap = plt.get_cmap('winter')
cmap = [cmap(x) for x in np.linspace(0, 1, len(name_and_energy))]
min_heuristic_energy = name_and_energy[0][1]
axins = inset_axes(ax, width="100%", height="100%",
                   bbox_to_anchor=(.21, .19, .65, .6),
                   bbox_transform=ax.transAxes, loc=2, borderpad=0)
for (name, energy), color in zip(name_and_energy, cmap):
    axins.axhline(-energy + annealing_energies[-1], color=color, linewidth=0.7)
axins.plot(total_evolution_times, -annealing_energies + annealing_energies[-1], '-ko', label = "GTQA")
axins.set_ylim(ymax=1)
axins.set_ylabel(r"$\Delta E$")
axins.set_xlabel(r"$T$")
axins.set_xscale("log")
axins.set_yscale("symlog")
plt.legend(frameon=False, fontsize=13, loc='lower center')
print(list(map(lambda x: x[0], name_and_energy)))
print(f"Best annealing objective function: {-min(annealing_energies)}, best heuristic objective function: {-min_heuristic_energy}")

In [2]:
fig_mqlib_hierarchy.savefig(f"{mqlib_results_dir_path}/mqlib_hierarchy.pdf",  bbox_inches="tight")